# 13Feb2020 Python analysis of tracks from Phenix for Serena Tsakali by George Chennell

Import relevant libraries

In [1]:
import pandas as pd
import numpy as np
import math

Import csv file with data avoiding the automatic idex column (index_col = False)

In [3]:
tracksframe = pd.read_csv('220719_tracks.csv', index_col=False)

is it the right shape?

In [4]:
tracksframe.shape

(17800, 22)

That looks correct

In [5]:
print(tracksframe.iloc[[3]])

   Row  Column  Plane  Field  Object No Compound  First Timepoint  \
3    6       7      1     19          4    20-FN                0   

   Last Timepoint Start Time  End Time  ...  Start Type  End Type Generation  \
3               6   00:00:00  00:37:32  ...       Begin    Cosmos          1   

  Parent ID  Root ID  Accumulated Distance [µm]  Displacement [µm]  \
3         4        4                    13.5066            7.97734   

   Speed [µm/s]  Displacement X [µm]  Displacement Y [µm]  
3      0.005996              1.79378              7.77305  

[1 rows x 22 columns]


This looks like our data (from row 3)

lets extract some useful information about the mean speed of all rows from a specific well..

In [6]:
print(tracksframe.columns)

Index(['Row', 'Column', 'Plane', 'Field', 'Object No', 'Compound',
       'First Timepoint', 'Last Timepoint', 'Start Time', 'End Time',
       'Number of Timepoints', 'Duration [s]', 'Start Type', 'End Type',
       'Generation', 'Parent ID', 'Root ID', 'Accumulated Distance [µm]',
       'Displacement [µm]', 'Speed [µm/s]', 'Displacement X [µm]',
       'Displacement Y [µm]'],
      dtype='object')


In [7]:
less = tracksframe.drop(columns = ['Plane', 'Field','Start Type', 'End Type','Generation', 'Parent ID', 'Root ID'])

so we can list all the indexes that have a condition 'Number of Timepoints' beneath or equal to 4

In [8]:
indexnames = less[less['Number of Timepoints'] < 5].index
indexnames.shape

(7953,)

In [9]:
less.drop(indexnames, inplace = True)

In [10]:
less.iloc[[2]]

,Row,Column,Object No,Compound,First Timepoint,Last Timepoint,Start Time,End Time,Number of Timepoints,Duration [s],Accumulated Distance [µm],Displacement [µm],Speed [µm/s],Displacement X [µm],Displacement Y [µm]
2,6,7,3,20-FN,0,6,00:00:00,00:37:32,7,2252.64,16.8893,9.10735,0.007498,-3.58756,8.37098


In [63]:
fltd = less[(less['Row']==3) & (less['Column'] == 3)]

In [64]:
fltd2 = fltd[(fltd['First Timepoint'] < 5)]

In [65]:
print(fltd['Speed [µm/s]'].mean())
print(fltd2['Speed [µm/s]'].mean())

0.009899087002989542
0.010176256494845357


In [66]:
ft3 = tracksframe[(tracksframe['Number of Timepoints'] >= 5)&(tracksframe['First Timepoint'] < 5) & (tracksframe['Compound'] == 'PBS')]
print('The mean speed of the cells in PBS with a track length longer than 5 timepoints & starting before the 5th time point is is', ft3['Speed [µm/s]'].mean(), 'µm/s')

The mean speed of the cells in PBS with a track length longer than 5 timepoints & starting before the 5th time point is is 0.0091214473519553 µm/s


In [11]:
compounds = tracksframe.Compound.unique()

In [12]:
compounds.tolist()

['20-FN',
 '15-FN',
 '5-FN+3-OMS',
 'PBS+3-OMS',
 'PBS',
 'FN+3-OMS',
 '20-FN+3-OMS',
 '5-FN',
 '15-FN+3-OMS',
 'FN',
 '10-FN+3-OMS',
 '10-FN']

In [13]:
print(compounds)

['20-FN' '15-FN' '5-FN+3-OMS' 'PBS+3-OMS' 'PBS' 'FN+3-OMS' '20-FN+3-OMS'
 '5-FN' '15-FN+3-OMS' 'FN' '10-FN+3-OMS' '10-FN']


To summarise the mean velocity of each compound

In [14]:
quants = ['Speed', 'StDev']
speeds = pd.DataFrame(index = compounds)
speed =[]
stdev =[]
counts = []
long =[]

print(speeds, "\n")
for x in compounds:
    ft4 = tracksframe[(tracksframe['Number of Timepoints'] >= 5)&(tracksframe['First Timepoint'] < 5) & (tracksframe['Compound'] == x)]
    thismean = ft4['Speed [µm/s]'].mean()
    howlong = ft4['Number of Timepoints'].mean()
    roundlong = round(howlong, 5)
    long.append(roundlong)
    thisstd = ft4['Speed [µm/s]'].std()
    thisround = round(thismean, 5)
    thissr = round(thisstd, 5)
    count = ft4['Speed [µm/s]'].count()
    counts.append(count)
    speed.append(thisround)
    stdev.append(thissr)
    print('the mean velocity of',count, 'cells in ',x,'is ', thisround, " µm/s with a standard deviation of ", thissr)

speeds['Count']=counts
speeds['Speeds µm/s']=speed
speeds['Speed StDev']=stdev
print(speeds)


Empty DataFrame
Columns: []
Index: [20-FN, 15-FN, 5-FN+3-OMS, PBS+3-OMS, PBS, FN+3-OMS, 20-FN+3-OMS, 5-FN, 15-FN+3-OMS, FN, 10-FN+3-OMS, 10-FN] 

the mean velocity of 247 cells in  20-FN is  0.01096  µm/s with a standard deviation of  0.00546
the mean velocity of 251 cells in  15-FN is  0.01043  µm/s with a standard deviation of  0.00439
the mean velocity of 298 cells in  5-FN+3-OMS is  0.00952  µm/s with a standard deviation of  0.00417
the mean velocity of 271 cells in  PBS+3-OMS is  0.00886  µm/s with a standard deviation of  0.00473
the mean velocity of 244 cells in  PBS is  0.00973  µm/s with a standard deviation of  0.00412
the mean velocity of 232 cells in  FN+3-OMS is  0.00912  µm/s with a standard deviation of  0.00517
the mean velocity of 277 cells in  20-FN+3-OMS is  0.01034  µm/s with a standard deviation of  0.00434
the mean velocity of 94 cells in  5-FN is  0.01098  µm/s with a standard deviation of  0.00477
the mean velocity of 230 cells in  15-FN+3-OMS is  0.0108  µm/s 

to Summarise the mean distance in each compound

In [15]:
distances = []
diststdev = []
for x in compounds:
    ft4 = tracksframe[(tracksframe['Number of Timepoints'] >= 5)&(tracksframe['First Timepoint'] < 5) & (tracksframe['Compound'] == x)]
    thismean = ft4['Accumulated Distance [µm]'].mean()
    thisstd = ft4['Accumulated Distance [µm]'].std()
    thisround = round(thismean, 5)
    distances.append(thisround)
    thissr = round(thisstd, 5)
    diststdev.append(thissr)
    print('the mean accumulated distance of cells in ',x,'is ', thisround, " µm with a standard deviation of ", thissr)

speeds['Distances (µm)'] = distances
speeds['Distance StDev'] = diststdev

print(speeds)

the mean accumulated distance of cells in  20-FN is  115.17776  µm with a standard deviation of  123.60264
the mean accumulated distance of cells in  15-FN is  107.51254  µm with a standard deviation of  123.77412
the mean accumulated distance of cells in  5-FN+3-OMS is  94.74998  µm with a standard deviation of  100.89535
the mean accumulated distance of cells in  PBS+3-OMS is  111.56306  µm with a standard deviation of  112.12111
the mean accumulated distance of cells in  PBS is  117.36109  µm with a standard deviation of  130.51744
the mean accumulated distance of cells in  FN+3-OMS is  108.45193  µm with a standard deviation of  111.02814
the mean accumulated distance of cells in  20-FN+3-OMS is  97.95659  µm with a standard deviation of  108.14721
the mean accumulated distance of cells in  5-FN is  113.21336  µm with a standard deviation of  108.62415
the mean accumulated distance of cells in  15-FN+3-OMS is  81.07965  µm with a standard deviation of  79.384
the mean accumulated d

speeds.to_csv(r'C:\Users\George Chennell\Documents\Python Scripts\pandas_analyse_csvs\analysisresults.csv')

In [16]:
speeds.to_csv(r'C:\Users\George Chennell\Documents\Python Scripts\pandas_analyse_csvs\220719_tracks_analysisresults.csv')
